# Steam 유사 / 경쟁 게임 분석 서비스

Notation
- 유사 게임 : 게임의 '태그'가 겹치는 게임으로서, 실제 두 게임이 얼마나 유사한지 측정
- 경쟁 게임 : 게임의 '유저'가 겹치는 게임으로서, 실제 두 게임의 유저가 얼마나 유사한지 측정

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from sqlalchemy import create_engine, text

## INIT

- DB 세팅
- MAGIC STRING 생성

In [2]:
sys.path.append('./config')
import config

DB_URL = config.auth_key['DB_URL']
ENGINE = create_engine(DB_URL)

## Load Data
크롤링으로 확보되어진 아래와 같은 데이터 셋에 대해 분석을 진행함

- df_applist : 스팀의 모든 '게임' 목록을 적재하고 있는 Steam_AppList 테이블 (DLC, 사운드트랙, 그 외 소프트웨어를 제외)
- df_taglist : 스팀 '게임' 상점 페이지 크롤링한 데이터에서 태그를 파싱후 적재한 데이터
- df_playlist : 유저가 계정 생성 시점으로부터 현재까지 플레이한 게임과 플레이타임 
- df_userlist : 유저의 상태 값

In [3]:
df_applist = pd.read_sql('select * from Steam.Steam_AppList', con = ENGINE)
print('df_applist (전체 게임 목록), shape : ', df_applist.shape)
display(df_applist.head())

df_taglist = pd.read_sql('select * from Steam.Steam_Tags', con = ENGINE)
print('df_taglist (게임 별 태그 목록), shape : ', df_taglist.shape)
display(df_taglist.head())

df_playlist = pd.read_sql('select * from Steam.Steam_Playtime', con = ENGINE)
print('df_applist (유저의 플레이 타임 데이터), shape : ', df_playlist.shape)
display(df_playlist.head())

df_userlist = pd.read_sql('select * from Steam.Steam_User', con = ENGINE)
print('df_applist (유저 정보 데이터), shape : ', df_userlist.shape)
display(df_userlist.head())

df_applist (전체 게임 목록), shape :  (108310, 5)


,appid,name,last_modified,price_change_number,CreateAt
0,10,Counter-Strike,1666823513,24324481,2024-07-17 00:00:47
1,20,Team Fortress Classic,1579634708,24324481,2024-07-17 00:00:47
2,30,Day of Defeat,1512413490,24324481,2024-07-17 00:00:47
3,40,Deathmatch Classic,1568752159,24324481,2024-07-17 00:00:47
4,50,Half-Life: Opposing Force,1579628243,24324481,2024-07-17 00:00:47


df_taglist (게임 별 태그 목록), shape :  (1415808, 6)


,tagid,name,count,browseable,appid,CreateAt
0,1663.0,FPS,649.0,1.0,300,2024-07-17 19:57:58
1,4150.0,World War II,515.0,1.0,300,2024-07-17 19:57:58
2,3859.0,Multiplayer,361.0,1.0,300,2024-07-17 19:57:58
3,19.0,Action,331.0,1.0,300,2024-07-17 19:57:58
4,1774.0,Shooter,307.0,1.0,300,2024-07-17 19:57:58


df_applist (유저의 플레이 타임 데이터), shape :  (564996, 4)


,appid,playtime_forever,playtime_2weeks,steamid
0,10,0,NaN,76561197960304592
1,10,1889,NaN,76561197960308272
2,10,0,NaN,76561197960335824
3,10,0,NaN,76561197960366192
4,10,0,NaN,76561197960448944


df_applist (유저 정보 데이터), shape :  (172535, 23)


,steamid,communityvisibilitystate,profilestate,personaname,profileurl,avatar,avatarmedium,avatarfull,avatarhash,personastate,...,commentpermission,loccountrycode,locstatecode,loccityid,realname,gameextrainfo,gameid,lobbysteamid,gameserverip,gameserversteamid
0,76561197960272880,3,1.0,edge,https://steamcommunity.com/profiles/7656119796...,https://avatars.steamstatic.com/94d878a103c451...,https://avatars.steamstatic.com/94d878a103c451...,https://avatars.steamstatic.com/94d878a103c451...,94d878a103c451032e37a3282af195cc9f561b90,0,...,NaN,DE,03,12679.0,None,None,None,None,None,None
1,76561197960282720,3,1.0,KIF,https://steamcommunity.com/profiles/7656119796...,https://avatars.steamstatic.com/0318d9324e908e...,https://avatars.steamstatic.com/0318d9324e908e...,https://avatars.steamstatic.com/0318d9324e908e...,0318d9324e908e5e6d7a5db96abbaa08fc4480ee,0,...,NaN,None,None,NaN,None,None,None,None,None,None
2,76561197960289808,3,1.0,kamikaze_decoy,https://steamcommunity.com/id/kamikaze_decoy/,https://avatars.steamstatic.com/fef49e7fa7e199...,https://avatars.steamstatic.com/fef49e7fa7e199...,https://avatars.steamstatic.com/fef49e7fa7e199...,fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb,0,...,NaN,GB,None,NaN,Carmen Campbell,None,None,None,None,None
3,76561197960293104,3,1.0,na$ty,https://steamcommunity.com/id/FiNN/,https://avatars.steamstatic.com/82383e334c4383...,https://avatars.steamstatic.com/82383e334c4383...,https://avatars.steamstatic.com/82383e334c4383...,82383e334c43835383b3f98fec211019f2c23332,0,...,1.0,US,CA,NaN,None,None,None,None,None,None
4,76561197960294096,1,1.0,dot,https://steamcommunity.com/profiles/7656119796...,https://avatars.steamstatic.com/1f0a6b1f646978...,https://avatars.steamstatic.com/1f0a6b1f646978...,https://avatars.steamstatic.com/1f0a6b1f646978...,1f0a6b1f6469782f709da26c4959f560d8cfe42f,0,...,NaN,None,None,NaN,None,None,None,None,None,None


## 유사도 알고리즘 정의

- 태그 기반 유사 게임 분석 : Jaccard / Tversky 유사도를 활용하여 태그가 얼마나 겹치는지에 대해 분석 (태그의 유/무 고려 O, 태그의 상관관계 고려 X) = 비교적 유사한 게임 도출하나, 유사한 태그를 골라내지 못함
    - Jaccard 유사도 함수
    - Tversky 유사도 함수

- 거리 기반 유사 게임 분석 : 태그를 Onehot 인코딩 후 데이터 간의 거리가 얼마나 먼지에 대한 분석 (태그의 유/무 고려 O, 태그 간 상관관계 고려 O) = 도메인적 판단시 특이한 게임을 도출하나, 데이터상 유사한 게임을 도출
    - Cosine 유사도 계산

In [4]:
# 자카드 유사도 계산 함수
def jaccard_similarity(features1, features2, *, use_weight = True):
    common_features = set(features1.keys()) & set(features2.keys())
    sum_min_weights = sum(min(features1[f], features2[f]) for f in common_features)
    sum_max_weights = sum(max(features1.get(f, 0), features2.get(f, 0)) for f in set(features1.keys()) | set(features2.keys()))

    if use_weight == True:
        return sum_min_weights / sum_max_weights if sum_max_weights else 0
    else:
        if len(set(features1.keys()).union(set(features2.keys()))) > 0:
            return len(set(features1.keys()).intersection(set(features2.keys()))) / len(set(features1.keys()).union(set(features2.keys()))) # 실제 정의
        else:
            return 0


# tversky 유사도 함수
# 특정 게임 -> feature1 / 측정 대상 게임 (개별 게임) -> feature2
def tversky_similarity(features1, features2, auto_params = True, *, use_weight = True, alpha = 0.5, beta = 0.5, param_threshold = 0.05):
    # reference : https://arxiv.org/pdf/2006.11372.pdf
    # alpha = beta = 0.5 : sorensen_dice_similarity
    # alpha = beta = 1 : jaccard_similarity
    if (alpha < 0) or (beta < 0):
        raise '0 <= Alpha, beta <= 1'
    elif (alpha > 1) or (beta > 1):
        raise '0 <= Alpha, beta <= 1'

    a_set = set(features1.keys())
    b_set = set(features2.keys())
    a_intersect_b = a_set.intersection(b_set)
    a_diff_b = a_set - b_set
    b_diff_a = b_set - a_set
    
    # 가중치 Auto : 태그가 많으면 개별 태그의 가중치 감소, 태그가 적은 게임의 영향을 강하게 탐지
    if auto_params == True:
        alpha = len(b_set) / (len(a_set) + len(b_set))
        beta = len(a_set) / (len(a_set) + len(b_set))
        
        # 최대 가중치 threshold -> 단일 태그를 너무 강하게 탐지 X
        if alpha < param_threshold:
            alpha, beta = param_threshold, 1-param_threshold
        elif beta < param_threshold:
            alpha, beta = 1-param_threshold, param_threshold
    
    weighted_a_intersect_b = sum([min(features1[f], features2[f]) for f in a_intersect_b])
    weighted_a_diff_b_set = sum([features1[f] for f in a_diff_b]) 
    weighted_b_diff_a_set = sum([features2[f] for f in b_diff_a]) 
    
    if use_weight == True:
        if (weighted_a_intersect_b + alpha*weighted_a_diff_b_set + beta*weighted_b_diff_a_set) > 0:
            return (weighted_a_intersect_b / (weighted_a_intersect_b + alpha*weighted_a_diff_b_set + beta*weighted_b_diff_a_set))
        else:
            return 0
    else:
        if (len(a_intersect_b) + (alpha * len(a_diff_b)) + (beta * len(b_diff_a))) > 0:
            return len(a_intersect_b) / (len(a_intersect_b) + (alpha * len(a_diff_b)) + (beta * len(b_diff_a)))
        else:
            return 0 # 실제 정의

## 유사 게임 연산 
- df_tag_dict : 태그 리스트에서 태그의 가중치를 부여하여 유사도를 연산하는 함수
    - apply문에서 특정 함수를 사용하여 특정 태그의 가중치를 상승시킬 수 있음

In [5]:
df_tag_dict = (df_taglist
               .groupby('appid')['tagid']
               .agg(list)
               .apply(lambda x : {i : 1 for i in x})
               )

display(df_tag_dict.head())
df_tag_dict = df_tag_dict.to_dict()

appid
10    {19.0: 1, 1663.0: 1, 3859.0: 1, 1774.0: 1, 169...
20    {19.0: 1, 1663.0: 1, 3859.0: 1, 1693.0: 1, 620...
30    {1663.0: 1, 4150.0: 1, 3859.0: 1, 1774.0: 1, 1...
40    {19.0: 1, 1663.0: 1, 1693.0: 1, 3859.0: 1, 177...
50    {1663.0: 1, 19.0: 1, 1693.0: 1, 3942.0: 1, 418...
Name: tagid, dtype: object

## 특정 게임에 대한 유사도 연산

- 특정 게임 설정

In [6]:
GAME_ID = 730

print('게임 ID에 해당하는 게임 정보')
display(df_applist[df_applist['appid'] == GAME_ID])

게임 ID에 해당하는 게임 정보


,appid,name,last_modified,price_change_number,CreateAt
24,730,Counter-Strike 2,1719426374,24324481,2024-07-17 00:00:47


### JACCARD 유사도 연산 결과

In [7]:
df_jaccard_sim = df_applist.copy()
df_jaccard_sim['sim_score'] = df_jaccard_sim['appid'].apply(lambda x : df_tag_dict[x] if x in df_tag_dict.keys() else {})
df_jaccard_sim['sim_score'] = df_jaccard_sim['sim_score'].apply(lambda x : jaccard_similarity(df_tag_dict[GAME_ID], x))
display(df_jaccard_sim.sort_values('sim_score', ascending=False).head(30))

,appid,name,last_modified,price_change_number,CreateAt,sim_score
24,730,Counter-Strike 2,1719426374,24324481,2024-07-17 00:00:47,1.000000
5330,359550,Tom Clancy's Rainbow Six® Siege,1719504498,24324481,2024-07-17 00:00:47,0.739130
37544,1218470,Lightphobe,1708970597,22411616,2024-07-17 00:00:47,0.600000
1634,222880,Insurgency,1610560212,24324481,2024-07-17 00:00:47,0.600000
10,240,Counter-Strike: Source,1718219165,24324481,2024-07-17 00:00:47,0.600000
50485,1539860,The Machines Arena,1666275231,0,2024-07-17 00:00:47,0.560000
365,15300,Tom Clancy's Ghost Recon®,1692818486,24324481,2024-07-17 00:00:47,0.541667
42005,1333540,Tactical Suppression,1592416965,11351614,2024-07-17 00:00:47,0.541667
16270,677620,Splitgate,1663343172,22736671,2024-07-17 00:00:47,0.538462
84728,2404300,Xyleron,1713746041,0,2024-07-17 00:00:47,0.538462


### Tversky 유사도 연산 결과

In [8]:
df_tversky_sim = df_applist.copy()
df_tversky_sim['sim_score'] = df_tversky_sim['appid'].apply(lambda x : df_tag_dict[x] if x in df_tag_dict.keys() else {})
df_tversky_sim['sim_score'] = df_tversky_sim['sim_score'].apply(lambda x : tversky_similarity(df_tag_dict[GAME_ID], x))
display(df_tversky_sim.sort_values('sim_score', ascending=False).head(30))

,appid,name,last_modified,price_change_number,CreateAt,sim_score
24,730,Counter-Strike 2,1719426374,24324481,2024-07-17 00:00:47,1.000000
5330,359550,Tom Clancy's Rainbow Six® Siege,1719504498,24324481,2024-07-17 00:00:47,0.850000
1634,222880,Insurgency,1610560212,24324481,2024-07-17 00:00:47,0.750000
37544,1218470,Lightphobe,1708970597,22411616,2024-07-17 00:00:47,0.750000
10,240,Counter-Strike: Source,1718219165,24324481,2024-07-17 00:00:47,0.750000
50485,1539860,The Machines Arena,1666275231,0,2024-07-17 00:00:47,0.718421
42005,1333540,Tactical Suppression,1592416965,11351614,2024-07-17 00:00:47,0.707353
365,15300,Tom Clancy's Ghost Recon®,1692818486,24324481,2024-07-17 00:00:47,0.707353
16270,677620,Splitgate,1663343172,22736671,2024-07-17 00:00:47,0.700000
84728,2404300,Xyleron,1713746041,0,2024-07-17 00:00:47,0.700000


In [9]:
# 이러면 간단하게 ONEHOT 인코딩 데이터프레임을 만들 수 있다.
df_tag_onehot = pd.DataFrame(df_tag_dict).T.fillna(0)
display(df_tag_onehot.head())

,19.0,1663.0,3859.0,1774.0,1693.0,5711.0,3839.0,3878.0,1708.0,6691.0,...,1445.0,143739.0,337964.0,603297.0,847164.0,1199779.0,117648.0,776177.0,1220528.0,189941.0
10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Cosine 유사도 연산 결과

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
df_cosine_sim = df_applist.copy()

for idx in tqdm(df_tag_onehot.index):
    df_cosine_sim.loc[df_cosine_sim['appid'] == idx, 'sim_score'] = cosine_similarity(df_tag_onehot.loc[GAME_ID].to_numpy().reshape(1, -1), df_tag_onehot.loc[idx].to_numpy().reshape(1, -1))
display(df_cosine_sim.sort_values('sim_score', ascending=False).head(30))

100%|██████████| 96010/96010 [00:59<00:00, 1618.14it/s]


,appid,name,last_modified,price_change_number,CreateAt,sim_score
24,730,Counter-Strike 2,1719426374,24324481,2024-07-17 00:00:47,1.000000
5330,359550,Tom Clancy's Rainbow Six® Siege,1719504498,24324481,2024-07-17 00:00:47,0.850000
10,240,Counter-Strike: Source,1718219165,24324481,2024-07-17 00:00:47,0.750000
1634,222880,Insurgency,1610560212,24324481,2024-07-17 00:00:47,0.750000
37544,1218470,Lightphobe,1708970597,22411616,2024-07-17 00:00:47,0.750000
50485,1539860,The Machines Arena,1666275231,0,2024-07-17 00:00:47,0.718185
365,15300,Tom Clancy's Ghost Recon®,1692818486,24324481,2024-07-17 00:00:47,0.705024
42005,1333540,Tactical Suppression,1592416965,11351614,2024-07-17 00:00:47,0.705024
84728,2404300,Xyleron,1713746041,0,2024-07-17 00:00:47,0.700000
1217,107410,Arma 3,1721131821,24324481,2024-07-17 00:00:47,0.700000


## 경쟁 게임 알고리즘

In [22]:
def get_cross_affinity(dataframe, target_game, *, target_col_name = 'appid', uid_col_name = 'steamid'):
    user_num = 0
    intersect_games = {}


    for uid, valdf in dataframe.groupby(uid_col_name):
        game_played = valdf[target_col_name].unique()
        user_num += 1
        
        if target_game in game_played:
            
            for cross_game in game_played:
                if cross_game in intersect_games.keys():
                    intersect_games[cross_game] += 1
                else:
                    intersect_games[cross_game] = 1

    
    df_game_proba = (dataframe.groupby(target_col_name).agg({uid_col_name : 'count'}) / user_num).reset_index().rename({uid_col_name : 'proba'}, axis = 1)

    df_cross_affinity = (pd.Series(intersect_games, name = 'support') / user_num).reset_index().rename({'index' : target_col_name}, axis = 1)
    
    # Target Game Proba
    df_cross_affinity['target_game_proba'] = df_game_proba[df_game_proba[target_col_name] == target_game]['proba'].values[0]
    
    # cross Game Proba
    df_cross_affinity = pd.merge(df_cross_affinity, df_game_proba, how = 'left', on = target_col_name).rename({'proba' : 'cross_game_proba'}, axis = 1)

    # 특정 게임을 즐기는 유저가, 해당 게임을 플레이할 확률
    df_cross_affinity['confidence(Penetration)'] = df_cross_affinity['support'] / df_cross_affinity['target_game_proba']
    # 일반 유저 대비 특정 게임을 즐기는 유저가 해당 게임을 플레이할 배율
    df_cross_affinity['lift(Affinity)'] = df_cross_affinity['support'] / (df_cross_affinity['target_game_proba'] * df_cross_affinity['cross_game_proba'])
    # 해당 게임을 즐기는 실제 유저 수
    df_cross_affinity['user_num'] = user_num * df_cross_affinity['cross_game_proba']
    
    # 자기 자신은 제외
    df_cross_affinity = df_cross_affinity[df_cross_affinity[target_col_name] != target_game]
    
    return df_cross_affinity

## 경쟁 게임 알고리즘 해석

- 타겟 게임 : 분석 대상 게임
- appid : Steam 상점의 Appid (= 교차 게임)
- support : 지지도 = 전체 유저 중 타겟 게임과 교차 게임을 함께 플레이하는 유저의 비중
- target_game_proba : 전체 유저 중 타겟 게임을 플레이하는 유저의 비중
- cross_game_proba : 전체 유저 중 교차 게임을 플레이하는 유저의 비중
- confidence(Penetration) : 타겟 게임을 즐기는 유저는 교차 게임을 N% 확률로 플레이 함
- Lift(Affinity) : 일반 유저 대비 특정 게임을 즐기는 유저가 해당 게임을 플레이할 배율
- user_num : 타겟 게임과 교차 게임을 함께 즐기는 유저 수 (표본 수)

In [34]:
display(pd.merge((get_cross_affinity(df_playlist, GAME_ID).sort_values('confidence(Penetration)', ascending = False)), df_applist.loc[:, ['appid', 'name']], how = 'left', on = 'appid').head(20))

,appid,support,target_game_proba,cross_game_proba,confidence(Penetration),lift(Affinity),user_num,name
0,578080,0.327959,0.710466,0.435241,0.461611,1.060587,6479.0,PUBG: BATTLEGROUNDS
1,1172470,0.201868,0.710466,0.226051,0.284134,1.256944,3365.0,Apex Legends™
2,431960,0.166398,0.710466,0.192261,0.234210,1.218184,2862.0,NaN
3,550,0.153030,0.710466,0.175265,0.215393,1.228956,2609.0,Left 4 Dead 2
4,1085660,0.123740,0.710466,0.145707,0.174168,1.195327,2169.0,Destiny 2
5,304930,0.123673,0.710466,0.137579,0.174073,1.265262,2048.0,Unturned
6,271590,0.117762,0.710466,0.134489,0.165753,1.232464,2002.0,Grand Theft Auto V
7,230410,0.116351,0.710466,0.132541,0.163767,1.235599,1973.0,Warframe
8,407530,0.114739,0.710466,0.136840,0.161498,1.180194,2037.0,ARK: The Survival Of The Fittest
9,346110,0.112723,0.710466,0.134556,0.158661,1.179146,2003.0,ARK: Survival Evolved
